In [3]:
import pickle
%load_ext autoreload
%autoreload 1

In [5]:
import os
import yaml
import sys

os.chdir('..')

In [6]:
%aimport src.ingest.get_bills
%aimport src.wrangle.create_features
%aimport src.analyze.run_model
%aimport src.report.store_db

In [7]:
from src.ingest.get_bills import get_us_bills
from src.ingest.get_bills import get_ny_bills
from src.ingest.get_bills import get_subjects
from src.wrangle.create_features import make_feat_union
from src.analyze.run_model import create_model
from src.analyze.run_model import run_model
from src.wrangle.create_features import make_x_values
from src.wrangle.create_features import make_y_values
from src.analyze.run_model import get_y_probs
from src.report.store_db import store_us_db
from src.report.store_db import store_ny_db

In [53]:
full_results = pickle.load(open('/Users/Joel/Desktop/Insight/bill_taxonomy/models/data_model_Taxation_2016-09-20-21-04-34.p'))

In [54]:
model, X, y = full_results

In [55]:
model.get_params()

{'cv': 3,
 'error_score': 'raise',
 'estimator': Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
        transformer_list=[('tfidf_title', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
         dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
         lowercase=True, max_df=0.4, max_features=None, min_df=10,
      ...ty='l1', random_state=None,
           solver='liblinear', tol=0.0001, verbose=0, warm_start=False))]),
 'estimator__features': FeatureUnion(n_jobs=1,
        transformer_list=[('tfidf_title', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
         dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
         lowercase=True, max_df=0.4, max_features=None, min_df=10,
         ngram_range=(1, 3), norm=u'l2',
         preproc...ol=0.1, random_state=0,
              topic_word_prior=None, total_samples=1000000.0, verbose=0))]))],
        transformer_weights=None),
 'estimator__features__lda_text

In [57]:
subject_name = 'Taxation'

In [ ]:
us_y_probs = model.predict_proba(X)[:, 1]

In [ ]:
from src.ingest.setup_database import US_Score
from src.ingest.setup_database import NY_Score
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine

In [ ]:
import psycopg2
import pandas as pd
# connect:
con = None
con = psycopg2.connect(database = 'bills_db', user = 'Joel')

# query:
sql_query = """
SELECT * FROM us_bills;
"""
us_bills = pd.read_sql_query(sql_query,con)

In [ ]:
import psycopg2
import pandas as pd
# connect:
con = None
con = psycopg2.connect(database = 'bills_db', user = 'Joel')

# query:
sql_query = """
SELECT * FROM ny_bills;
"""
ny_bills = pd.read_sql_query(sql_query,con)

In [ ]:
len(ny_bills)

In [ ]:
X_ny = make_x_values(ny_bills)

In [ ]:
ny_y_probs = model.predict_proba(X_ny)[:, 1]

In [ ]:
len(ny_y_probs)

In [ ]:
engine = create_engine('postgres://%s@localhost/%s' % ('Joel', 'bills_db'))

# Open a session and connect to the database engine
Session = sessionmaker(bind=engine)
session = Session()

for i, bill in enumerate(ny_y_probs):

    one_bill = NY_Score(subject=subject_name, bill_num=ny_bills.ix[i, 'bill_num'],
                        score=ny_y_probs[i])
    session.add(one_bill)
session.commit()
session.close()

In [ ]:
engine = create_engine('postgres://%s@localhost/%s' % ('Joel', 'bills_db'))

# Open a session and connect to the database engine
Session = sessionmaker(bind=engine)
session = Session()

for i, bill in enumerate(us_y_probs):

    one_bill = US_Score(subject=subject_name, bill_num=us_bills.ix[i, 'bill_num'],
                        actual=bool(y[i]), score=us_y_probs[i])
    session.add(one_bill)
session.commit()
session.close()